<a href="https://colab.research.google.com/github/datamaki/Arduino-101/blob/main/C%C3%B3pia_de_Transfer_Learning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================================
# 0) GPU e versões
# =========================================================
!nvidia-smi -L || echo "Sem GPU (ok para teste, mas treina mais lento)"
import tensorflow as tf, sys, os, numpy as np
print("TF:", tf.__version__)


/bin/bash: line 1: nvidia-smi: command not found
Sem GPU (ok para teste, mas treina mais lento)
TF: 2.19.0


In [ ]:
# Setup básico (rode ANTES de criar a EfficientNet)
import tensorflow as tf
from tensorflow.keras import layers, models
# import robusto da EfficientNet (funciona em várias versões do TF/Keras)
try:
    from tensorflow.keras.applications import EfficientNetB0
except:
    from tensorflow.keras.applications.efficientnet import EfficientNetB0

IMG_SIZE = (224, 224)   # agora existe :)
BATCH = 32


In [ ]:
# =========================================================
# 1) Dataset via TFDS (recomendado)
#    -> Já vem limpo dos arquivos corrompidos e com labels
# =========================================================
!pip -q install -U tensorflow-datasets
import tensorflow_datasets as tfds
(ds_train, ds_val, ds_test), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True, as_supervised=True
)
num_classes = info.features['label'].num_classes
print(info)

IMG_SIZE = (224, 224)
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_img(img, label):
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

# Augmentation leve (só no train)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
])

def add_aug(img, label):
    img = data_augmentation(img, training=True)
    return img, label

train_ds = ds_train.map(preprocess_img, num_parallel_calls=AUTOTUNE)\
                   .map(add_aug, num_parallel_calls=AUTOTUNE)\
                   .batch(BATCH).prefetch(AUTOTUNE)

val_ds   = ds_val.map(preprocess_img, num_parallel_calls=AUTOTUNE)\
                 .batch(BATCH).prefetch(AUTOTUNE)

test_ds  = ds_test.map(preprocess_img, num_parallel_calls=AUTOTUNE)\
                  .batch(BATCH).prefetch(AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.6P4XDC_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings 

In [ ]:
# =========================================================
# 2) Modelo: EfficientNetB0 + cabeça nova
#    Fase 1: extrair características (base congelada)
# =========================================================
from tensorflow.keras import layers, models
try:
    from tensorflow.keras.applications import EfficientNetB0
except:
    from tensorflow.keras.applications.efficientnet import EfficientNetB0

IMG_SIZE = (224, 224)

base = EfficientNetB0(include_top=False, weights='imagenet',
                      input_shape=IMG_SIZE + (3,))
base.trainable = False

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

2) Treino (fase 1) + fine-tuning

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

ckpt = ModelCheckpoint('/content/effb0_best.h5', monitor='val_accuracy', save_best_only=True)
es   = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
rlr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

history1 = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[ckpt, es, rlr])

# fine-tuning: destrava só o "final" da base
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy', metrics=['accuracy'])
history2 = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[ckpt, es, rlr])


Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9571 - loss: 0.1489

582/582 ━━━━━━━━━━━━━━━━━━━━ 1534s 3s/step - accuracy: 0.9572 - loss: 0.1488 - val_accuracy: 0.9897 - val_loss: 0.0324 - learning_rate: 0.0010
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9865 - loss: 0.0385

582/582 ━━━━━━━━━━━━━━━━━━━━ 1532s 3s/step - accuracy: 0.9865 - loss: 0.0385 - val_accuracy: 0.9910 - val_loss: 0.0281 - learning_rate: 0.0010
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1588s 3s/step - accuracy: 0.9885 - loss: 0.0329 - val_accuracy: 0.9910 - val_loss: 0.0255 - learning_rate: 0.0010
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9903 - loss: 0.0299

582/582 ━━━━━━━━━━━━━━━━━━━━ 1523s 3s/step - accuracy: 0.9903 - loss: 0.0299 - val_accuracy: 0.9914 - val_loss: 0.0244 - learning_rate: 0.0010
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9889 - loss: 0.0312

582/582 ━━━━━━━━━━━━━━━━━━━━ 1513s 3s/step - accuracy: 0.9889 - loss: 0.0312 - val_accuracy: 0.9918 - val_loss: 0.0245 - learning_rate: 0.0010
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1502s 3s/step - accuracy: 0.9896 - loss: 0.0303 - val_accuracy: 0.9918 - val_loss: 0.0242 - learning_rate: 0.0010
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1541s 3s/step - accuracy: 0.9892 - loss: 0.0297 - val_accuracy: 0.9914 - val_loss: 0.0250 - learning_rate: 0.0010
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9910 - loss: 0.0258

582/582 ━━━━━━━━━━━━━━━━━━━━ 1534s 3s/step - accuracy: 0.9910 - loss: 0.0258 - val_accuracy: 0.9923 - val_loss: 0.0257 - learning_rate: 0.0010
Epoch 9/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1537s 3s/step - accuracy: 0.9904 - loss: 0.0268 - val_accuracy: 0.9918 - val_loss: 0.0243 - learning_rate: 5.0000e-04
Epoch 10/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 1529s 3s/step - accuracy: 0.9912 - loss: 0.0242 - val_accuracy: 0.9923 - val_loss: 0.0243 - learning_rate: 5.0000e-04
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1797s 3s/step - accuracy: 0.9587 - loss: 0.1071 - val_accuracy: 0.9910 - val_loss: 0.0286 - learning_rate: 1.0000e-05
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1720s 3s/step - accuracy: 0.9773 - loss: 0.0583 - val_accuracy: 0.9910 - val_loss: 0.0268 - learning_rate: 1.0000e-05
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1709s 3s/step - accuracy: 0.9814 - loss: 0.0484 - val_accuracy: 0.9905 - val_loss: 0.0260 - learning_rate: 1.0000e-05
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1721s 3s/step - accuracy: 0.98

3) Avaliar + matriz de confusão

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_probs = model.predict(val_ds).ravel()
val_pred  = (val_probs > 0.5).astype(int)
y_true    = np.concatenate([y.numpy() for _, y in val_ds.unbatch().batch(2048)], axis=0)

val_acc = model.evaluate(val_ds, verbose=0)[1]
test_acc = model.evaluate(test_ds, verbose=0)[1]
print(f"Val acc: {val_acc:.4f}  |  Test acc: {test_acc:.4f}")

print("\nConfusion matrix:\n", confusion_matrix(y_true, val_pred))
print("\nReport:\n", classification_report(y_true, val_pred, target_names=['cat','dog']))


73/73 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step
Val acc: 0.9910  |  Test acc: 0.9927

Confusion matrix:
 [[1109   11]
 [  10 1196]]

Report:
               precision    recall  f1-score   support

         cat       0.99      0.99      0.99      1120
         dog       0.99      0.99      0.99      1206

    accuracy                           0.99      2326
   macro avg       0.99      0.99      0.99      2326
weighted avg       0.99      0.99      0.99      2326



4) Salvar .h5 e .tflite

In [ ]:
model.save('/content/effb0_transfer.h5')

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # quantização dinâmica
tflite = converter.convert()
open('/content/effb0_transfer.tflite','wb').write(tflite)


Saved artifact at '/tmp/tmpkgyqlfj_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_242')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139884292212880: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139884292215184: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139884274226320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274225552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274227664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274226704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274227280: TensorSpec(shape=(), dtype=tf.resource, name=

4548072

In [ ]:
import os, tensorflow as tf
os.makedirs("/content/models", exist_ok=True)

# 1) Salvar no formato Keras (recomendado)
model.save("/content/models/effb0_transfer.keras")   # ✅

# 2) (opcional) Também em HDF5, se precisar de compatibilidade antiga
model.save("/content/models/effb0_transfer.h5")

# 3) Exportar TFLite com quantização dinâmica
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite = converter.convert()
open("/content/models/effb0_transfer.tflite","wb").write(tflite)

!ls -lh /content/models


Saved artifact at '/tmp/tmpq6v2_xzs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_242')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139884292212880: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139884292215184: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139884274226320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274225552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274227664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274228048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274226704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139884274227280: TensorSpec(shape=(), dtype=tf.resource, name=

copiar pro Drive:

In [ ]:
from google.colab import drive; drive.mount("/content/drive")
!cp /content/models/* /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
import glob, tensorflow as tf, os

print("Procurando modelos…")
cands = glob.glob("/content/**/*.keras", recursive=True) + glob.glob("/content/**/*.h5", recursive=True)
for p in cands: print(" -", p)

if any(p.endswith(".keras") for p in cands):
    path = next(p for p in cands if p.endswith(".keras"))
    m = tf.keras.models.load_model(path)
    print("Carregado:", path)
elif any(p.endswith(".h5") for p in cands):
    path = next(p for p in cands if p.endswith(".h5"))
    m = tf.keras.models.load_model(path)      # checkpoint completo em .h5
    print("Carregado:", path)
    os.makedirs("/content/models", exist_ok=True)
    m.save("/content/models/effb0_transfer.keras")   # re-salva no formato novo
    print("Re-salvo em /content/models/effb0_transfer.keras")
else:
    raise FileNotFoundError("Não encontrei .keras nem .h5 em /content")


Procurando modelos…
 - /content/models/effb0_transfer.keras
 - /content/drive/MyDrive/effb0_transfer.keras
 - /content/effb0_transfer.h5
 - /content/effb0_best.h5
 - /content/models/effb0_transfer.h5
 - /content/drive/MyDrive/effb0_transfer.h5
 - /content/drive/MyDrive/classificacao_flor_girassol/modelo_flor_girassol.h5
 - /content/drive/MyDrive/classificacao_flor_girassol/modelo_flor_girassol_v2.h5
Carregado: /content/models/effb0_transfer.keras


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 32 variables whereas the saved optimizer has 62 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


carregar

In [ ]:
import tensorflow as tf
m = tf.keras.models.load_model("effb0_transfer.keras")


ValueError: File not found: filepath=effb0_transfer.keras. Please ensure the file is an accessible `.keras` zip file.